In [6]:
pwd

'/home/sh/lab/etritf'

In [2]:
cd /home/sh/lab/etritf

/home/sh/lab/etritf


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline

In [94]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing

# 2020년
- 기준: deepsleepduration 예측
- 데이터: last.csv
- 예측변수: 

In [48]:
df_2020 = pd.read_csv('./ja_data/last.csv')
df_2020.head()

,Unnamed: 0,userId,date,endInput,sleep,sleepProblem,dream,amCondition,amEmotion,pmEmotion,...,hr_average,hr_min,hr_max,rr_average,rr_min,rr_max,breathing_disturbances_intensity,snoring,snoringepisodecount,sleep_score
0,0,user01,2020-08-30,2020-08-31 8:43,3.0,6.0,2.0,3.0,3.0,4.0,...,69,57,83,16,12,25,18,8880,16,83
1,1,user01,2020-08-31,2020-09-01 10:18,2.0,3.0,2.0,2.0,3.0,4.0,...,73,63,82,16,12,22,20,8280,16,82
2,2,user01,2020-09-02,2020-09-03 11:04,3.0,3.0,2.0,2.0,3.0,4.0,...,72,64,80,16,11,24,20,11400,11,82
3,3,user01,2020-09-03,2020-09-04 12:33,4.0,3.0,2.0,3.0,3.0,2.0,...,72,60,85,16,12,21,14,5280,5,60
4,4,user01,2020-09-04,2020-09-05 10:25,3.0,3.0,4.0,2.0,3.0,4.0,...,71,61,82,16,11,22,15,5460,11,79


In [49]:
df_2020.columns

Index(['Unnamed: 0', 'userId', 'date', 'endInput', 'sleep', 'sleepProblem',
       'dream', 'amCondition', 'amEmotion', 'pmEmotion', 'pmStress',
       'pmFatigue', 'caffeine', 'cAmount(ml)', 'alcohol', 'aAmount(ml)',
       'startDt', 'endDt', 'lastUpdate', 'wakeupduration',
       'lightsleepduration', 'deepsleepduration', 'wakeupcount',
       'durationtosleep', 'remsleepduration', 'durationtowakeup', 'hr_average',
       'hr_min', 'hr_max', 'rr_average', 'rr_min', 'rr_max',
       'breathing_disturbances_intensity', 'snoring', 'snoringepisodecount',
       'sleep_score'],
      dtype='object')

In [64]:
#다중 분류를 위해 y값 변환
labels = ['Very Poor', 'Poor', 'Fair', 'Good', 'Excellent']
score_classes = pd.qcut(df_2020['deepsleepduration'], q=5, labels=labels, duplicates='drop')

## score_classes를 데이터프레임의 새로운 열로 추가
df_2020['score_classes'] = score_classes
print(df_2020.value_counts('score_classes'))

## 라벨 인코딩
le = preprocessing.LabelEncoder()
df_2020['score_classes'] = le.fit_transform(df_2020['score_classes'])


score_classes
Poor         102
Fair         102
Very Poor    101
Good         100
Excellent     99
dtype: int64


In [65]:
X = df_2020[['wakeupduration','lightsleepduration', 'durationtosleep', 'remsleepduration', 'durationtowakeup', 'hr_average',
       'hr_min', 'snoringepisodecount']]
y = df_2020['score_classes']

In [72]:
X = df_2020[['wakeupduration','lightsleepduration', 'durationtosleep', 'remsleepduration', 'durationtowakeup', 'hr_average',
       'hr_min', 'snoringepisodecount']]
y = df_2020['score_classes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred,average='weighted')
rf_recall = recall_score(y_test, rf_y_pred,average='weighted')
rf_f1 = f1_score(y_test, rf_y_pred,average='weighted')
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")
# Decision Tree 모델
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_y_pred)
dt_precision = precision_score(y_test, dt_y_pred,average='weighted')
dt_recall = recall_score(y_test, dt_y_pred,average='weighted')
dt_f1 = f1_score(y_test, dt_y_pred,average='weighted')
print("Decision Tree Accuracy:", dt_accuracy)
print("Decision Tree Precision:", dt_precision)
print("Decision Tree Recall:", dt_recall)
print("Decision Tree F1-score:", dt_f1)
print("---------------------------------------")
# Gradient Boosting 모델
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb_y_pred = gb.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred,average='weighted')
gb_recall = recall_score(y_test, gb_y_pred,average='weighted')
gb_f1 = f1_score(y_test, gb_y_pred,average='weighted')
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Gradient Boosting Precision:", gb_precision)
print("Gradient Boosting Recall:", gb_recall)
print("Gradient Boosting F1-score:", gb_f1)
print("---------------------------------------")
# Support Vector Machine 모델
svm = SVC()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred,average='weighted')
svm_recall = recall_score(y_test, svm_y_pred,average='weighted')
svm_f1 = f1_score(y_test, svm_y_pred,average='weighted')
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Support Vector Machine Precision:", svm_precision)
print("---------------------------------------")
#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred,average='weighted')
xgb_recall = recall_score(y_test, xgb_y_pred,average='weighted')
xgb_f1 = f1_score(y_test, xgb_y_pred,average='weighted')
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)

Random Forest Accuracy: 0.40594059405940597
Random Forest Precision: 0.4577677138910646
Random Forest Recall: 0.40594059405940597
Random Forest F1-score: 0.4155303051575719
---------------------------------------
Decision Tree Accuracy: 0.31683168316831684
Decision Tree Precision: 0.35453758045487804
Decision Tree Recall: 0.31683168316831684
Decision Tree F1-score: 0.3176884440690539
---------------------------------------
Gradient Boosting Accuracy: 0.3564356435643564
Gradient Boosting Precision: 0.43857790600045465
Gradient Boosting Recall: 0.3564356435643564
Gradient Boosting F1-score: 0.36532133814132356
---------------------------------------
Support Vector Machine Accuracy: 0.31683168316831684
Support Vector Machine Precision: 0.3355335533553356
---------------------------------------


/home/sh/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB Accuracy: 0.4158415841584158
XGB Precision: 0.5052085568376927
XGB Recall: 0.4158415841584158
XGB F1-score: 0.4303092883987801


In [ ]:
X = df_2020[[ 'wakeupduration','lightsleepduration', 'durationtosleep', 'remsleepduration', 'durationtowakeup', 'hr_average',
       'hr_min', 'snoringepisodecount']]
y = df_2020['deepsleepduration']

In [60]:
# 이진 분류를 위해 y값 변환
y_binary = y.apply(lambda x: 1 if x >= 5370 else 0)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)


# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred)
rf_recall = recall_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")
# Decision Tree 모델
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_y_pred)
dt_precision = precision_score(y_test, dt_y_pred)
dt_recall = recall_score(y_test, dt_y_pred)
dt_f1 = f1_score(y_test, dt_y_pred)
print("Decision Tree Accuracy:", dt_accuracy)
print("Decision Tree Precision:", dt_precision)
print("Decision Tree Recall:", dt_recall)
print("Decision Tree F1-score:", dt_f1)
print("---------------------------------------")
# Gradient Boosting 모델
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb_y_pred = gb.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred)
gb_recall = recall_score(y_test, gb_y_pred)
gb_f1 = f1_score(y_test, gb_y_pred)
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Gradient Boosting Precision:", gb_precision)
print("Gradient Boosting Recall:", gb_recall)
print("Gradient Boosting F1-score:", gb_f1)
print("---------------------------------------")
# Support Vector Machine 모델
svm = SVC()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred)
svm_recall = recall_score(y_test, svm_y_pred)
svm_f1 = f1_score(y_test, svm_y_pred)
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Support Vector Machine Precision:", svm_precision)
print("---------------------------------------")
#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred)
xgb_recall = recall_score(y_test, xgb_y_pred)
xgb_f1 = f1_score(y_test, xgb_y_pred)
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)


Random Forest Accuracy: 0.7623762376237624
Random Forest Precision: 0.72
Random Forest Recall: 0.782608695652174
Random Forest F1-score: 0.7499999999999999
---------------------------------------
Decision Tree Accuracy: 0.693069306930693
Decision Tree Precision: 0.6829268292682927
Decision Tree Recall: 0.6086956521739131
Decision Tree F1-score: 0.6436781609195402
---------------------------------------
Gradient Boosting Accuracy: 0.7623762376237624
Gradient Boosting Precision: 0.7391304347826086
Gradient Boosting Recall: 0.7391304347826086
Gradient Boosting F1-score: 0.7391304347826085
---------------------------------------
Support Vector Machine Accuracy: 0.6435643564356436
Support Vector Machine Precision: 0.5806451612903226
---------------------------------------
XGB Accuracy: 0.7623762376237624
XGB Precision: 0.7391304347826086
XGB Recall: 0.7391304347826086
XGB F1-score: 0.7391304347826085


# 2019년
- 기준: waso 1200
- ‘time_in_bed’, ‘wakeupcount’, ‘movement_index’, ‘sleep_frag_index’

In [73]:
data=pd.read_csv('./ja_data/total1819.csv').drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
data=data.fillna(-1)
data19=data[data['device']=='Actigraph']

In [74]:
data19.head()

,userId,night_inputDt,pmEmotion,pmStress,ifUnusual,breakfast,lunch,dinner,lateSnack,amCaffeine,...,device,sleep_score,total_sleep_time,time_in_bed,waso,wakeupcount,aal,movement_index,fragmentation_index,sleep_frag_index
217,102,2019-12-15 22:16,4.0,5.0,1.0,3.0,3.0,3.0,3.0,-1,...,Actigraph,82.44,26760,32460,5700.0,3.0,31.67,6.412,25.0,31.412
218,102,2019-12-15 22:16,4.0,5.0,1.0,3.0,3.0,3.0,3.0,-1,...,Actigraph,97.89,36120,36900,780.0,2.0,6.50,12.060,0.0,12.060
219,104,2019-12-04 23:55,2.0,2.0,2.0,3.0,2.0,4.0,4.0,-1,...,Actigraph,96.48,16440,17040,600.0,8.0,1.25,4.930,0.0,4.930
220,104,2019-12-04 23:55,2.0,2.0,2.0,3.0,2.0,4.0,4.0,-1,...,Actigraph,98.03,32880,33540,660.0,4.0,2.75,1.431,0.0,1.431
221,104,2019-12-05 23:41,3.0,3.0,2.0,3.0,3.0,4.0,4.0,tea,...,Actigraph,99.53,25320,25440,120.0,2.0,1.00,2.358,0.0,2.358


In [75]:
data19.columns

Index(['userId', 'night_inputDt', 'pmEmotion', 'pmStress', 'ifUnusual',
       'breakfast', 'lunch', 'dinner', 'lateSnack', 'amCaffeine',
       'amCaffAmount', 'pmCaffeine', 'pmCaffAmount', 'alcohol', 'aAmount',
       'date', 'sleep', 'inputDt', 'sleepProblem', 'dream', 'amCondition',
       'amEmotion', 'total_CaffAmount', 'date_m', 'startDt', 'endDt', 'device',
       'sleep_score', 'total_sleep_time', 'time_in_bed', 'waso', 'wakeupcount',
       'aal', 'movement_index', 'fragmentation_index', 'sleep_frag_index'],
      dtype='object')

In [76]:
#다중 분류를 위해 y값 변환
labels = ['Very Poor', 'Poor', 'Fair', 'Good', 'Excellent']
score_classes = pd.qcut(data19['waso'], q=5, labels=labels, duplicates='drop')

## score_classes를 데이터프레임의 새로운 열로 추가
data19['score_classes'] = score_classes
print(data19.value_counts('score_classes'))

## 라벨 인코딩
le = preprocessing.LabelEncoder()
data19['score_classes'] = le.fit_transform(data19['score_classes'])

score_classes
Very Poor    63
Good         57
Fair         56
Excellent    56
Poor         54
dtype: int64


/tmp/ipykernel_1317956/703695039.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data19['score_classes'] = score_classes
/tmp/ipykernel_1317956/703695039.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data19['score_classes'] = le.fit_transform(data19['score_classes'])


In [77]:
X = data19[['time_in_bed', 'wakeupcount', 'movement_index', 'sleep_frag_index']]
y = data19['score_classes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred,average='weighted')
rf_recall = recall_score(y_test, rf_y_pred,average='weighted')
rf_f1 = f1_score(y_test, rf_y_pred,average='weighted')
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")
# Decision Tree 모델
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_y_pred)
dt_precision = precision_score(y_test, dt_y_pred,average='weighted')
dt_recall = recall_score(y_test, dt_y_pred,average='weighted')
dt_f1 = f1_score(y_test, dt_y_pred,average='weighted')
print("Decision Tree Accuracy:", dt_accuracy)
print("Decision Tree Precision:", dt_precision)
print("Decision Tree Recall:", dt_recall)
print("Decision Tree F1-score:", dt_f1)
print("---------------------------------------")
# Gradient Boosting 모델
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb_y_pred = gb.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred,average='weighted')
gb_recall = recall_score(y_test, gb_y_pred,average='weighted')
gb_f1 = f1_score(y_test, gb_y_pred,average='weighted')
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Gradient Boosting Precision:", gb_precision)
print("Gradient Boosting Recall:", gb_recall)
print("Gradient Boosting F1-score:", gb_f1)
print("---------------------------------------")
# Support Vector Machine 모델
svm = SVC()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred,average='weighted')
svm_recall = recall_score(y_test, svm_y_pred,average='weighted')
svm_f1 = f1_score(y_test, svm_y_pred,average='weighted')
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Support Vector Machine Precision:", svm_precision)
print("---------------------------------------")
#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred,average='weighted')
xgb_recall = recall_score(y_test, xgb_y_pred,average='weighted')
xgb_f1 = f1_score(y_test, xgb_y_pred,average='weighted')
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)

Random Forest Accuracy: 0.7931034482758621
Random Forest Precision: 0.8335199283475146
Random Forest Recall: 0.7931034482758621
Random Forest F1-score: 0.7915425480537043
---------------------------------------
Decision Tree Accuracy: 0.7241379310344828
Decision Tree Precision: 0.746572249589491
Decision Tree Recall: 0.7241379310344828
Decision Tree F1-score: 0.7208929641939142
---------------------------------------
Gradient Boosting Accuracy: 0.7931034482758621
Gradient Boosting Precision: 0.8460446247464503
Gradient Boosting Recall: 0.7931034482758621
Gradient Boosting F1-score: 0.7937221865257847
---------------------------------------
Support Vector Machine Accuracy: 0.4827586206896552
Support Vector Machine Precision: 0.3279074748340115
---------------------------------------


/home/sh/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB Accuracy: 0.7586206896551724
XGB Precision: 0.784667487684729
XGB Recall: 0.7586206896551724
XGB F1-score: 0.760895341802783


In [79]:
X = data19[['time_in_bed', 'wakeupcount', 'movement_index', 'sleep_frag_index']]
y = data19['waso']

In [80]:
# 이진 분류를 위해 y값 변환
y_binary = y.apply(lambda x: 1 if x >= 1200 else 0)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)


# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred)
rf_recall = recall_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")
# Decision Tree 모델
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_y_pred)
dt_precision = precision_score(y_test, dt_y_pred)
dt_recall = recall_score(y_test, dt_y_pred)
dt_f1 = f1_score(y_test, dt_y_pred)
print("Decision Tree Accuracy:", dt_accuracy)
print("Decision Tree Precision:", dt_precision)
print("Decision Tree Recall:", dt_recall)
print("Decision Tree F1-score:", dt_f1)
print("---------------------------------------")
# Gradient Boosting 모델
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb_y_pred = gb.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred)
gb_recall = recall_score(y_test, gb_y_pred)
gb_f1 = f1_score(y_test, gb_y_pred)
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Gradient Boosting Precision:", gb_precision)
print("Gradient Boosting Recall:", gb_recall)
print("Gradient Boosting F1-score:", gb_f1)
print("---------------------------------------")
# Support Vector Machine 모델
svm = SVC()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred)
svm_recall = recall_score(y_test, svm_y_pred)
svm_f1 = f1_score(y_test, svm_y_pred)
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Support Vector Machine Precision:", svm_precision)
print("---------------------------------------")
#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred)
xgb_recall = recall_score(y_test, xgb_y_pred)
xgb_f1 = f1_score(y_test, xgb_y_pred)
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)


Random Forest Accuracy: 0.9655172413793104
Random Forest Precision: 1.0
Random Forest Recall: 0.9375
Random Forest F1-score: 0.967741935483871
---------------------------------------
Decision Tree Accuracy: 0.9482758620689655
Decision Tree Precision: 0.967741935483871
Decision Tree Recall: 0.9375
Decision Tree F1-score: 0.9523809523809523
---------------------------------------
Gradient Boosting Accuracy: 0.9655172413793104
Gradient Boosting Precision: 0.96875
Gradient Boosting Recall: 0.96875
Gradient Boosting F1-score: 0.96875
---------------------------------------
Support Vector Machine Accuracy: 0.8103448275862069
Support Vector Machine Precision: 0.7837837837837838
---------------------------------------
XGB Accuracy: 0.9655172413793104
XGB Precision: 0.96875
XGB Recall: 0.96875
XGB F1-score: 0.96875


# 2018년
- 기준: sleepProblem 1
- ‘오후에 받은 스트레스’, ‘수면 주관평가점수’, ‘저녁식사 유무’

In [85]:
data=pd.read_csv('./ja_data/total1819.csv').drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)
data=data.fillna(-1)
data18=data[data['device']=='Fitbit']
data18=data18.drop(['waso', 'wakeupcount','aal','movement_index','fragmentation_index', 'sleep_frag_index','userId', 'night_inputDt', 'amCaffeine','pmCaffeine','alcohol','device','inputDt','date','date_m','startDt', 'endDt'],axis=1)

In [86]:
data18.head()

,pmEmotion,pmStress,ifUnusual,breakfast,lunch,dinner,lateSnack,amCaffAmount,pmCaffAmount,aAmount,sleep,sleepProblem,dream,amCondition,amEmotion,total_CaffAmount,sleep_score,total_sleep_time,time_in_bed
0,3.0,1.0,2.0,1.0,3.0,3.0,4.0,-1.0,-1.0,-1.0,3.0,3.0,2.0,4.0,5.0,-1.0,90.800478,22800,25110
1,5.0,4.0,2.0,1.0,3.0,3.0,-1.0,2.0,-1.0,-1.0,3.0,0.0,4.0,3.0,4.0,-1.0,91.984231,21000,22830
2,3.0,2.0,2.0,2.0,3.0,3.0,4.0,-1.0,1.0,-1.0,4.0,6.0,4.0,2.0,2.0,-1.0,88.482385,19590,22140
3,4.0,2.0,2.0,2.0,3.0,3.0,4.0,1.0,1.0,-1.0,3.0,0.0,4.0,1.0,5.0,2.0,88.781665,22080,24870
4,3.0,2.0,2.0,1.0,3.0,3.0,2.0,-1.0,-1.0,-1.0,4.0,0.0,4.0,1.0,2.0,-1.0,92.403101,17880,19350


In [87]:
data18.columns

Index(['pmEmotion', 'pmStress', 'ifUnusual', 'breakfast', 'lunch', 'dinner',
       'lateSnack', 'amCaffAmount', 'pmCaffAmount', 'aAmount', 'sleep',
       'sleepProblem', 'dream', 'amCondition', 'amEmotion', 'total_CaffAmount',
       'sleep_score', 'total_sleep_time', 'time_in_bed'],
      dtype='object')

In [90]:
#다중 분류를 위해 y값 변환
labels = ['Fair', 'Good', 'Excellent']
score_classes = pd.qcut(data18['sleep_score'], q=3, labels=labels, duplicates='drop')

## score_classes를 데이터프레임의 새로운 열로 추가
data18['score_classes'] = score_classes
print(data18.value_counts('score_classes'))

le = preprocessing.LabelEncoder()
data18['score_classes'] = le.fit_transform(data18['score_classes'])

score_classes
Fair         73
Good         72
Excellent    72
dtype: int64


In [91]:
X = data18[['pmStress','dinner', 'sleep']]
y = data18['score_classes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred,average='weighted')
rf_recall = recall_score(y_test, rf_y_pred,average='weighted')
rf_f1 = f1_score(y_test, rf_y_pred,average='weighted')
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")
# Decision Tree 모델
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_y_pred)
dt_precision = precision_score(y_test, dt_y_pred,average='weighted')
dt_recall = recall_score(y_test, dt_y_pred,average='weighted')
dt_f1 = f1_score(y_test, dt_y_pred,average='weighted')
print("Decision Tree Accuracy:", dt_accuracy)
print("Decision Tree Precision:", dt_precision)
print("Decision Tree Recall:", dt_recall)
print("Decision Tree F1-score:", dt_f1)
print("---------------------------------------")
# Gradient Boosting 모델
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb_y_pred = gb.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred,average='weighted')
gb_recall = recall_score(y_test, gb_y_pred,average='weighted')
gb_f1 = f1_score(y_test, gb_y_pred,average='weighted')
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Gradient Boosting Precision:", gb_precision)
print("Gradient Boosting Recall:", gb_recall)
print("Gradient Boosting F1-score:", gb_f1)
print("---------------------------------------")
# Support Vector Machine 모델
svm = SVC()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred,average='weighted')
svm_recall = recall_score(y_test, svm_y_pred,average='weighted')
svm_f1 = f1_score(y_test, svm_y_pred,average='weighted')
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Support Vector Machine Precision:", svm_precision)
print("---------------------------------------")
#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred,average='weighted')
xgb_recall = recall_score(y_test, xgb_y_pred,average='weighted')
xgb_f1 = f1_score(y_test, xgb_y_pred,average='weighted')
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)

Random Forest Accuracy: 0.3409090909090909
Random Forest Precision: 0.44554924242424243
Random Forest Recall: 0.3409090909090909
Random Forest F1-score: 0.36189855544694255
---------------------------------------
Decision Tree Accuracy: 0.36363636363636365
Decision Tree Precision: 0.41385281385281386
Decision Tree Recall: 0.36363636363636365
Decision Tree F1-score: 0.381948051948052
---------------------------------------
Gradient Boosting Accuracy: 0.3409090909090909
Gradient Boosting Precision: 0.4685482646008962
Gradient Boosting Recall: 0.3409090909090909
Gradient Boosting F1-score: 0.3444375507953094
---------------------------------------
Support Vector Machine Accuracy: 0.3181818181818182
Support Vector Machine Precision: 0.5942234848484849
---------------------------------------
XGB Accuracy: 0.3409090909090909
XGB Precision: 0.4643143621084798
XGB Recall: 0.3409090909090909
XGB F1-score: 0.34034430324752907


In [92]:
X = data18[['pmStress','dinner', 'sleep']]
y = data18['sleepProblem']

In [93]:
# 이진 분류를 위해 y값 변환
y_binary = y.apply(lambda x: 1 if x >= 1 else 0)

# 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)


# Random Forest 모델
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
rf_precision = precision_score(y_test, rf_y_pred)
rf_recall = recall_score(y_test, rf_y_pred)
rf_f1 = f1_score(y_test, rf_y_pred)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Precision:", rf_precision)
print("Random Forest Recall:", rf_recall)
print("Random Forest F1-score:", rf_f1)
print("---------------------------------------")
# Decision Tree 모델
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_y_pred)
dt_precision = precision_score(y_test, dt_y_pred)
dt_recall = recall_score(y_test, dt_y_pred)
dt_f1 = f1_score(y_test, dt_y_pred)
print("Decision Tree Accuracy:", dt_accuracy)
print("Decision Tree Precision:", dt_precision)
print("Decision Tree Recall:", dt_recall)
print("Decision Tree F1-score:", dt_f1)
print("---------------------------------------")
# Gradient Boosting 모델
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
gb_y_pred = gb.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
gb_precision = precision_score(y_test, gb_y_pred)
gb_recall = recall_score(y_test, gb_y_pred)
gb_f1 = f1_score(y_test, gb_y_pred)
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Gradient Boosting Precision:", gb_precision)
print("Gradient Boosting Recall:", gb_recall)
print("Gradient Boosting F1-score:", gb_f1)
print("---------------------------------------")
# Support Vector Machine 모델
svm = SVC()
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)
svm_precision = precision_score(y_test, svm_y_pred)
svm_recall = recall_score(y_test, svm_y_pred)
svm_f1 = f1_score(y_test, svm_y_pred)
print("Support Vector Machine Accuracy:", svm_accuracy)
print("Support Vector Machine Precision:", svm_precision)
print("---------------------------------------")
#xgb 모델
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_precision = precision_score(y_test, xgb_y_pred)
xgb_recall = recall_score(y_test, xgb_y_pred)
xgb_f1 = f1_score(y_test, xgb_y_pred)
print("XGB Accuracy:", xgb_accuracy)
print("XGB Precision:", xgb_precision)
print("XGB Recall:", xgb_recall)
print("XGB F1-score:", xgb_f1)


Random Forest Accuracy: 0.6818181818181818
Random Forest Precision: 0.68
Random Forest Recall: 0.7391304347826086
Random Forest F1-score: 0.7083333333333334
---------------------------------------
Decision Tree Accuracy: 0.6590909090909091
Decision Tree Precision: 0.6666666666666666
Decision Tree Recall: 0.6956521739130435
Decision Tree F1-score: 0.6808510638297872
---------------------------------------
Gradient Boosting Accuracy: 0.5681818181818182
Gradient Boosting Precision: 0.5714285714285714
Gradient Boosting Recall: 0.6956521739130435
Gradient Boosting F1-score: 0.6274509803921569
---------------------------------------
Support Vector Machine Accuracy: 0.6363636363636364
Support Vector Machine Precision: 0.6521739130434783
---------------------------------------
XGB Accuracy: 0.5454545454545454
XGB Precision: 0.5517241379310345
XGB Recall: 0.6956521739130435
XGB F1-score: 0.6153846153846154
